In [204]:
import random
from pulp import *
from collections import deque
import time
from collections import Counter
import networkx as nx
import math

# Ucitavanje grafova

In [205]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph

def loadGraph(input_file):
    global d
    G = nx.Graph()
    for j in range(0,n):
        G.add_node(j)

    f = open(input_file, "r")
    string = f.readline()
    string = f.readline()
    string = f.readline()
    for i in range(0, m):
        string = f.readline()
        string = string.split()
        j = int(string[0])-1
        k = int(string[1])-1
        G.add_edge(j, k)
    f.close()
    d = []
    j = 1
    for i in range(n):
        d.append([float("inf")]*(j))
        j+=1
    for i in range(G.number_of_nodes()):
        ds = nx.single_source_shortest_path_length(G, i)
        for key in ds:
            if i >= key:
                d[i][key] = ds[key]
            else:
                d[key][i] = ds[key]
    print("n: " + str(G.number_of_nodes()))
    print("m: " + str(G.number_of_edges()))
    print("ncc: " + str(nx.number_connected_components(G)))


In [206]:
def all_pairs_distance_matrix(graph):
    """
    Calculate all-pairs shortest distances using BFS for each node.
    :param graph: dict where keys are nodes (1 to n) and values are lists of neighbors
    :return: 2D list (matrix) where element [i][j] is the shortest distance from node i+1 to j+1
    """
    nodes = sorted(graph.keys())  # Sortiraj čvorove kako bi odgovarali indeksima matrice
    n = len(nodes)
    distance_matrix = [[math.inf] * n for _ in range(n)]  # Inicijalizuj matricu sa beskonačnostima

    for start in nodes:
        visited = {node: False for node in nodes}
        d_local = {node: math.inf for node in nodes}
        q = deque()
        d_local[start] = 0
        visited[start] = True
        q.append(start)

        while q:
            current = q.popleft()
            for neighbor in graph.get(current, []):
                if not visited[neighbor]:
                    visited[neighbor] = True
                    q.append(neighbor)
                    d_local[neighbor] = d_local[current] + 1

        # Popuni red u matrici udaljenosti za čvor `start`
        for node in nodes:
            distance_matrix[start - 1][node - 1] = d_local[node]

    return distance_matrix

def valid_burning_sequence(s, n, d):
    """
    Validate if the given burning sequence is valid for a graph of size n.
    :param s: List of burning nodes (sequence of nodes).
    :param n: Number of nodes in the graph.
    :param d: Distance matrix (2D list) where d[i][j] represents the shortest distance between node i+1 and j+1.
    :return: True if the sequence is valid, otherwise False.
    """
    counter = 0

    for i in range(n):
        b = len(s)
        for j in range(b):
            if d[i][s[j] - 1] <= b - (j + 1):  # Adjust index for 1-based node numbering
                counter += 1
                break

    return counter == n

# Generisanje nasumicnih burning sekvenci

In [207]:
def find_best_node(graph, NB, B):
    """
    Pronalaženje čvora koji pokriva najviše neprekrivenih čvorova.
    """
    max_covered = -1
    best_node = None

    for node in NB:
        # Prebrojavanje neprekrivenih suseda trenutnog čvora
        uncovered_neighbors = sum(1 for neighbor in graph[node] if neighbor not in B)
        if uncovered_neighbors > max_covered:
            max_covered = uncovered_neighbors
            best_node = node

    return best_node


def generate_random_burning_sequence_with_greedy(graph):
    """
    Generisanje sekvence sagorevanja sa smanjenom nasumičnošću pomoću grešnog pristupa.
    """
    S = []  # sekvenca zapaljenih čvorova
    B = set()  # skup zapaljenih čvorova
    NB = set(graph.keys())  # skup nezapaljenih čvorova, inicijalno su to svi čvorovi grafa

    # Prvi čvor se bira nasumično
    current_node = random.choice(list(NB))
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)

    while len(B) < len(graph):
        # Sagorevanje suseda trenutnog čvora
        neighbors_to_burn = [neighbor for neighbor in graph[current_node] if neighbor in NB]
        for neighbor in neighbors_to_burn:
            B.add(neighbor)
            NB.remove(neighbor)

        # Pronalaženje najboljeg sledećeg čvora
        best_node = find_best_node(graph, NB, B)

        if best_node is not None:
            # Dodavanje najboljeg čvora u sekvencu
            S.append(best_node)
            B.add(best_node)
            NB.remove(best_node)
            current_node = best_node  # Ažuriramo trenutni čvor
        else:
            # Ako nema "najboljeg" čvora, biramo nasumičan čvor
            if NB:
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
                current_node = random_node
                # Ažuriramo trenutni čvor
            else:
                break
    return S


def generate_multiple_random_burning_sequences_with_greedy(graph, num_sequences):
    """
    Generisanje više sekvenci sagorevanja sa smanjenom nasumičnošću.
    """
    sequences = []
    for _ in range(num_sequences):
        sequence = generate_random_burning_sequence_with_greedy(graph)
        if sequence not in sequences and valid_burning_sequence(sequence, len(graph), all_pairs_distance_matrix(graph)):
            sequences.append(sequence)
    return sequences


# Učitavanje grafa
filename = "grafovi/manji/rt-retweet.mtx"
graph = load_graph_from_mtx(filename)

# Generisanje sekvenci
num_sequences = 50
burning_sequences1 = generate_multiple_random_burning_sequences_with_greedy(graph, num_sequences)

# Ispis rezultata
print(f"Generisane burning sekvence:")
for seq in burning_sequences1:
    print(seq, "len:", len(seq))


Generisane burning sekvence:
[45, 54, 53, 5, 41, 15, 28, 72, 48, 50, 13, 18, 63, 65, 2, 3, 4, 7, 23, 26, 30, 34, 42, 46, 49, 52, 55, 59, 60, 68, 71, 76, 80, 82, 85, 86, 90, 94, 96] len: 39
[91, 54, 45, 53, 5, 41, 15, 28, 72, 48, 50, 13, 18, 63, 2, 3, 4, 7, 23, 26, 30, 34, 42, 46, 49, 52, 55, 59, 60, 68, 71, 76, 80, 82, 85, 86, 90, 94, 96] len: 39
[17, 54, 45, 53, 41, 15, 28, 72, 48, 50, 13, 18, 40, 63, 65, 2, 3, 4, 7, 16, 20, 23, 26, 30, 34, 42, 46, 49, 52, 55, 60, 68, 71, 76, 80, 82, 85, 86, 90, 94, 96] len: 41
[86, 54, 45, 53, 5, 41, 15, 28, 72, 48, 50, 13, 18, 63, 65, 2, 3, 4, 7, 23, 26, 30, 34, 42, 46, 49, 52, 55, 59, 60, 68, 71, 76, 80, 82, 85, 90, 94, 96] len: 39
[58, 54, 56, 53, 5, 41, 15, 28, 13, 48, 50, 74, 18, 63, 65, 71, 3, 4, 7, 23, 26, 30, 34, 35, 42, 44, 46, 49, 55, 59, 62, 68, 69, 76, 82, 83, 85, 86, 94, 96] len: 40
[80, 54, 45, 53, 5, 41, 15, 28, 72, 48, 50, 13, 18, 63, 65, 2, 3, 4, 7, 23, 26, 30, 34, 42, 46, 49, 52, 55, 59, 60, 68, 71, 76, 82, 85, 86, 90, 94, 96] len: 

In [208]:
def transpose_list_of_lists(input_list):
    """
    Transponuje listu listi tako da prvi elementi svih podlisti budu u prvoj listi, drugi u drugoj, itd.
    """
    transposed = []
    for item in zip(*input_list):
        unique_item_list = list(set(item))
        transposed.append(unique_item_list)
    return transposed

burning_sequences_t = transpose_list_of_lists(burning_sequences1)
print(burning_sequences_t)


[[1, 2, 4, 5, 8, 14, 17, 19, 21, 22, 25, 28, 31, 35, 36, 40, 42, 45, 48, 50, 52, 55, 58, 65, 69, 70, 73, 74, 77, 80, 81, 84, 86, 88, 91, 92, 94, 96], [45, 54], [72, 45, 53, 56, 89], [89, 53, 5], [41, 53, 5], [41, 53, 15], [28, 53, 15], [72, 41, 15, 48, 50, 28], [72, 13, 15, 48, 22, 28, 61], [1, 4, 13, 48, 50, 31], [48, 50, 18, 13], [4, 74, 13, 18, 50, 23, 63], [65, 4, 40, 74, 13, 18, 29, 30, 63], [65, 2, 13, 49, 50, 18, 30, 63], [65, 2, 3, 63], [65, 2, 3, 4, 71], [2, 3, 4, 71, 7], [3, 4, 7, 10, 23], [4, 7, 10, 12, 16, 18, 23, 26], [7, 10, 16, 17, 18, 23, 25, 26, 30], [34, 18, 20, 23, 26, 30], [32, 34, 39, 23, 26, 27, 30], [33, 34, 39, 42, 26, 30], [34, 35, 36, 42, 44, 46, 30], [34, 36, 37, 42, 46, 49], [37, 38, 42, 44, 46, 49, 52], [38, 42, 46, 47, 49, 52, 55], [42, 47, 49, 52, 55, 57, 59], [47, 49, 52, 55, 59, 60], [66, 68, 49, 52, 55, 59, 60, 62], [68, 71, 52, 55, 59, 60, 62], [68, 69, 71, 76, 55, 59, 60], [69, 71, 76, 80, 59, 60, 62], [67, 76, 77, 78, 80, 82, 60, 63], [67, 68, 80, 8

# PuLP

In [209]:
    
def ILP(L,k):
    global d  
    
    #model
    model = LpProblem("ILP_COV", LpMinimize)
    
    # Definišemo varijable
  

# Definišemo b[j] varijable
    b = [LpVariable("b_{}".format(j+1), cat='Binary') for j in range(n)]
    x = [[LpVariable("x_{}_{}".format(r+1, i+1), cat='Binary') for i in range(n+1)] for r in range(k)]
    
    s1= 0
    for i in range(k):
        for j in range(n+1):
            s1 += x[i][j]
    model+=s1
        
    
    #domensko ogranicenje
    for i in range(int(k/2)):
        model += lpSum(x[i][j] for j in burning_sequences_t[i]) == 1

    
    #ogranicenja
    #(13)
    for i in range(k):
        suma=0
        for j in range(n):
            suma+=x[i][j]
        model +=(suma<=1)
        
    #(14)
    for j in range(n):
        suma = 0
        for i in range(k):
            for kk in range(n):
                if kk >= j:
                    if d[kk][j] <= i:
                        suma += x[i][kk]
                else:
                    if d[j][kk] <= i:
                        suma += x[i][kk]
        model +=(b[j]<=suma)
        
    #(12)
    for i in range(k):
        sum2 = 0
        if i == 0:
            sum1 = 1
        else:
            sum1 = 0
        for j in range(n):
            sum2 += x[i][j]
            if i >= 1:
                sum1 += x[i-1][j]
        model +=(sum2<=sum1)

    
    #(15)
    model +=lpSum(b[j] for j in range(n)) == n 
    
    
    # rjesavanje
    model.solve(PULP_CBC_CMD(maxSeconds=600, msg=True, fracGap=0))
    '''for i in range(k):
        for j in range(n+1):
            if x[i][j].value() == 1:
                print(f"x[{i}, {j}] = 1")
            else:
                print(f"x[{i}, {j}] = 0")'''


    if LpStatus[model.status] == 'Optimal':
        print(f"Optimal Solution Found: {model.objective.value()}")
        for i in range(k):
            for j in range(n):
                if x[i][j].value() == 1:
                    print(f"x[{i}, {j}] = 1")
    else:
        print("No optimal solution found")
    s = []
    
    x_out = [[0]*n for i in range(k)]
    for r in range(k):
        i = 0
        for e in x_out[r]:
            if e == 1:
                s.append(i)
                break
            i += 1
 
    s.reverse()
   

    return s
        
def main(ni, mi, input_file, Li, Ui):
    n = ni
    m = mi
    U = Ui
    L = Li
    loadGraph(input_file)
    s = ILP(L,U)

if __name__ == "__main__":
    bs = []
    n = 0
    m = 0
    d = {}
    bs_size = float("inf")
    folder_dataset = 'grafovi/manji/'
    dataset = [
        #['karate.mtx',34,78,2,3] #['dolphins.mtx',62,159,2,4],
        #['chesapeake.mtx',39,170,1,3], # instance, n, m, l, h
        #['dolphins.mtx',62,159,2,4],
        ['rt-retweet.mtx',96,117,2,5],
        #['polbooks.mtx',105,441,2,4],
        #['adjnoun.mtx',112,425,2,4],
        #['ia-infect-hyper.mtx',113,2196,1,3],
        #['C125-9.mtx',125,6963,1,3],
        #['ia-enron-only.mtx',143,623,2,4],
        #['c-fat200-1.mtx',200,1534,3,7],
        #['c-fat200-2.mtx',200,3235,2,5],
        #['c-fat200-5.mtx',200,8473,1,3],
        ]
    for i in range(len(dataset)):
        print("________________________________________________")
        instance = dataset[i][0]
        print("instance: " + instance)
        n = dataset[i][1]
        m = dataset[i][2]
        L = dataset[i][3]
        U = dataset[i][4]
        main(n, m, folder_dataset + dataset[i][0], L, U)
    
    

________________________________________________
instance: rt-retweet.mtx
n: 96
m: 117
ncc: 1
Optimal Solution Found: 5.0
x[0, 48] = 1
x[1, 45] = 1
x[2, 64] = 1
x[3, 74] = 1
x[4, 53] = 1
